# electricity_market_env

> Fill in a module description here

In [ ]:
#| default_exp electricity_market_env

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from enum import Enum

import gymnasium as gym
import numpy as np
from gymnasium.envs.registration import register


In [ ]:
#| export
class Weather(Enum):
    SUNNY = 1
    CLOUDY = 2
    PARTIAL_CLOUDY = 4

WEATHER_PROBABILITIES_MAP = {
    Weather.SUNNY: 0.5,
    Weather.CLOUDY: 0.3,
    Weather.PARTIAL_CLOUDY: 0.2
}

WEATHER_OPTIONS, WEATHER_PROBABILITIES = zip(*WEATHER_PROBABILITIES_MAP.items())

In [ ]:
#| export
class ElectricityMarketEnv(gym.Env):
    def __init__(self, env_config=None):
        # Environment Configuration
        if env_config is None:
            env_config = {}
        # State of Environment
        self._timestep = 0
        self._init_state_of_charge = env_config.get('init_state_of_charge', 50)
        self._current_state_of_charge = self._init_state_of_charge
        self._battery_capacity = env_config.get('battery_capacity', 100)
        self._production_capacity = env_config.get('production_capacity', 50)
        self._max_timestep = env_config.get('max_timestep', 10_000)
        self._battery_efficiency = 0.95
        self._battery_degradation = 0.999
        self._config = env_config
        self.__weather = self._get_weather()
        self.action_space = gym.spaces.Discrete(2 * self._battery_capacity + 1)
        self.actions = list(range(-self._battery_capacity, self._battery_capacity + 1))
        # observation space: (Battery SoC, Current electricity demand, Current market price)
        self.observation_space = gym.spaces.Box(low=np.array([0, 0, 0]),
                                                high=np.array([self._battery_capacity, np.inf, np.inf]),
                                                shape=(3,), dtype=np.float64)


    def _is_action_valid(self, charge_amount) -> bool:
        if charge_amount > self._production:
            return False
        target_state_of_charge = self._current_state_of_charge + charge_amount * self._battery_efficiency

        return 0 <= target_state_of_charge <= self._battery_capacity

    def step(self, action):
        charge_amount = self.actions[action]
        done = self._battery_efficiency == 0 or self._timestep >= self._max_timestep
        truncated = False
        if not self._is_action_valid(charge_amount):
            print("Action {} is not valid".format(action))
            truncated = True
            return self._get_obs(), 0, done, truncated, {}
        self._current_state_of_charge += charge_amount * self._battery_efficiency

        # self._battery_capacity *= self._battery_degradation
        reward = self._reward(charge_amount)
        observations = self._get_obs()
        self._timestep += 1
        self.__weather = self._get_weather()
        return observations, reward, done, truncated, {}

    @staticmethod
    def _get_weather():
        return np.random.choice(WEATHER_OPTIONS, p=WEATHER_PROBABILITIES)

    @property
    def _weather(self):
        return self.__weather

    @property
    def _production(self):
        match self._weather:
            case Weather.SUNNY:
                return self._production_capacity
            case Weather.CLOUDY:
                return self._production_capacity * 0.4
            case Weather.PARTIAL_CLOUDY:
                return self._production_capacity * 0.7
            case _:
                raise ValueError("Weather not supported")

    def _reward(self, charge_amount):
        if charge_amount > 0:
            reward = 0
        else:
            reward = (abs(charge_amount) - self._demand_of_electricity) * self._sell_price
        if self._current_state_of_charge < 20 or self._current_state_of_charge > 80:
            reward *= 0.9

        return reward

    @property
    def _demand_of_electricity(self):
        base_demand = 1 + 50 * np.sin(self._timestep * 2 * np.pi)  # Cyclical demand
        noise = np.random.normal(0, 10)  # Random noise
        return abs(base_demand + noise)

    @property
    def _buy_price(self):
        ...
        return None

    @property
    def _sell_price(self):
        """ Simulates electricity price changes based on demand and supply. """
        base_price = 100 + 50 * np.sin(self._timestep * 2 * np.pi)  # Cyclical price changes
        noise = np.random.normal(0, 5)  # Random price fluctuations
        return abs(base_price + noise)

    def reset(self, *, seed=None, options=None):
        """ Resets the environment to the initial state. """
        super().reset(seed=seed)
        self._timestep = 0
        self._init_state_of_charge = self._config.get('init_state_of_charge', 50)
        self._current_state_of_charge = self._init_state_of_charge
        self._battery_capacity = self._config.get('battery_capacity', 100)
        self._battery_efficiency = 0.95
        self._battery_degradation = 0.999
        self.__weather = self._get_weather()
        return self._get_obs(), {}

    def action_masks(self):
        """Generate a boolean mask of valid actions for `MaskablePPO`."""
        mask = np.array([self._is_action_valid(self.actions[i]) for i in range(self.action_space.n)], dtype=bool)
        if not np.any(mask):  # If all actions are invalid, force one to be valid
            mask[np.random.randint(len(mask) // 2)] = True
        return mask

    def _get_obs(self):
        """ Returns the current observation (state). """
        return np.array([
            self._current_state_of_charge,  # Battery SoC
            self._demand_of_electricity,  # Current electricity demand
            self._sell_price  # Current market price
        ])



In [ ]:
#| hide
register(
    id='ElectricityMarketEnv-v0',
    entry_point='electricity_market.electricity_market_env:ElectricityMarketEnv',
)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()